<a href="https://colab.research.google.com/github/abhi-1907/TrainingSessionsForStudents/blob/main/DNNnew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Rescaling
import matplotlib.pyplot as plt
import os
import numpy as np
from google.colab import files
import os
import zipfile

In [14]:

# Upload the ZIP file (Manually select your "handwriting.zip")
uploaded = files.upload()


Saving handwriting.zip to handwriting.zip


In [17]:
import zipfile
import os
import shutil

zip_filename = "/content/handwriting.zip"
extract_to = "/content/handwriting/"  # Correct extraction path

# Ensure the target directory is empty before extraction
if os.path.exists(extract_to):
    shutil.rmtree(extract_to)

# Extract ZIP file
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("✅ Dataset extracted to:", extract_to)

# Verify extraction
!ls -R /content/handwriting/



✅ Dataset extracted to: /content/handwriting/
/content/handwriting/:
test  train

/content/handwriting/test:
0  1  2  3  4  5  6  7	8  9

/content/handwriting/test/0:
digit_0_ccc06c12-1951-11e9-b486-309c2384bdbc.jpg  digit_0_ccf22a66-1951-11e9-8621-309c2384bdbc.jpg
digit_0_ccc4ffde-1951-11e9-ab0d-309c2384bdbc.jpg  digit_0_ccf3ff2e-1951-11e9-90f1-309c2384bdbc.jpg
digit_0_ccc5ea46-1951-11e9-8892-309c2384bdbc.jpg  digit_0_ccf7f6b6-1951-11e9-9100-309c2384bdbc.jpg
digit_0_ccef6b46-1951-11e9-bafb-309c2384bdbc.jpg  digit_0_ccf81dcc-1951-11e9-a37b-309c2384bdbc.jpg
digit_0_ccf0a3ca-1951-11e9-9680-309c2384bdbc.jpg  digit_0_ccf9f29a-1951-11e9-82a8-309c2384bdbc.jpg

/content/handwriting/test/1:
digit1__f6553500-bd9b-11e7-8d1c-327da8d6bd0e.jpg  digit1__fceb933c-5d4c-11e7-a1da-86a857f0136a.jpg
digit1__f8156930-8a8a-11e7-a46b-66ea4b620b09.jpg  digit1__fda84132-be16-11e7-93bf-de3f7c15671c.jpg
digit1__faa7ffe6-8aa3-11e7-aa75-32ad16d2437a.jpg  digit1__fdb21a70-c5d9-11e7-a0bd-864837419dd5.jpg
digit1__fab

In [18]:
base_dir = "/content/handwriting/"
train_dir = os.path.join(base_dir, "train")
test_dir = os.path.join(base_dir, "test")

if os.path.exists(train_dir):
    print("✅ Train directory found!")
else:
    print("❌ Train directory NOT found!")

if os.path.exists(test_dir):
    print("✅ Test directory found!")
else:
    print("❌ Test directory NOT found!")


✅ Train directory found!
✅ Test directory found!


In [19]:
print("TensorFlow Version:", tf.__version__)

TensorFlow Version: 2.18.0


In [20]:
# Image parameters
img_height = 28
img_width = 28
batch_size = 32

# Load training dataset
train_ds = keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(img_height, img_width),
    color_mode="grayscale",
    batch_size=batch_size,
    label_mode="categorical"
)

# Load test dataset
test_ds = keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=(img_height, img_width),
    color_mode="grayscale",
    batch_size=batch_size,
    label_mode="categorical"
)

# Optimize dataset loading
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

print("Dataset loaded successfully!")



Found 307 files belonging to 10 classes.
Found 133 files belonging to 10 classes.
Dataset loaded successfully!


In [21]:
model = keras.Sequential([
    Rescaling(1.0 / 255, input_shape=(img_height, img_width, 1)),  # Normalize images
    Flatten(),  # Convert images to a 1D array
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')  # Output layer for 10 digit classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)                │ (None, 28, 28, 1)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         100,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 101,770 (397.54 KB)

 Trainable params: 101,770 (397.54 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
# Train the model for 5 epochs
model.fit(train_ds, epochs=25)



Epoch 1/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.3850 - loss: 1.7602
Epoch 2/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.4317 - loss: 1.6608
Epoch 3/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4693 - loss: 1.6043
Epoch 4/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4550 - loss: 1.5953
Epoch 5/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.4706 - loss: 1.5273
Epoch 6/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4911 - loss: 1.4886
Epoch 7/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.5063 - loss: 1.4330
Epoch 8/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5682 - loss: 1.3949
Epoch 9/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4663 - loss: 1.4074
Epoch 10/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5913 - loss: 1.4062
Epoch 11/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5996 - loss: 1.3399
Epoch 12/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy:

In [26]:
# Evaluate the model on the test dataset
loss, accuracy = model.evaluate(test_ds, verbose=1)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.4564 - loss: 1.7063
Test Loss: 1.7515369653701782
Test Accuracy: 0.4135338366031647


In [27]:
# Get model predictions
predictions = model.predict(test_ds)

# Convert predictions to class labels
predicted_labels = tf.argmax(predictions, axis=1).numpy()
print("Predicted labels:", predicted_labels)


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted labels: [4 6 2 6 4 1 6 4 6 4 1 5 8 4 6 4 6 9 1 9 4 6 0 9 1 9 1 1 6 6 1 4 9 0 1 1 9
 2 9 8 6 9 9 1 1 9 4 2 6 8 1 6 9 6 1 4 8 9 2 1 9 6 4 8 9 6 6 1 4 1 6 2 6 7
 0 1 6 9 1 4 9 8 9 4 4 6 8 9 6 9 0 1 0 9 0 6 1 4 4 9 9 6 4 1 0 2 0 5 6 6 4
 4 9 1 5 1 0 4 6 2 4 9 2 2 6 4 9 2 1 6 1 2 1]
